# TEST NER Custom con Spacy

Instalamos librerías necesarias

In [39]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install spacy[transformers]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 9.0 MB/s 
     |████████████████████████████████| 1.2 MB 61.4 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
numba 0.56.2 requires setuptools<60, but you have setuptools 65.3.0 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


cargamos los datos de entrenamiento

In [20]:
import json
 
with open('/content/data.json', 'r') as f:
    data = json.load(f)
    
print(data['annotations'][0])

['CANCELLATIONS\r\nANY TIME\r\nCHARGE USD 300.00 FOR CANCEL.\r\nNOTE  TEXT BELOW NOT VALIDATED FOR AUTOPRICING.\r\n\r\nFOR ALL JOURNEYS BEFORE AFTER FIRST FLIGHT\r\nDEPARTURE  USD300\r\n\r\nAPPLICABLE FEES ARE TO BE CONVERTED INTO\r\nLOCAL CURRENCY AT THE BANKERS SELLING RATE.\r\n\r\nFEES ARE NOT APPLICABLE FOR INFANT WITHOUT A SEAT\r\n\r\nIN CASE OF TICKET UPGRADE THE ORIGINAL\r\nNONREFUNDABLE AMOUNT REMAINS NONREFUNDABLE.\r\n\r\nYR IS NONREFUNDABLE FOR ALL FARES.\r\n\r\nIN CASES OF NOSHOW CHARGE NO SHOW FEE OF USD300\r\nAND CANCELLATION FEE OF USD300 FOR CANCEL.\r\n\r\nFOR ANY CANCELLATIONS THE STRICTER FARE\r\nCONDITIONS WILL APPLY TO THE ENTIRE JOURNEY.\r\n\r\n\r\n\r\n', {'entities': [[0, 23, 'CANCELLATION_TIME'], [25, 54, 'CHARGE_CANCEL'], [458, 486, 'CHARGE_NOSHOW']]}]


organizamos los datos para que sean input del modelo

In [21]:

dict_temp = {}
training_data = []
for line in data["annotations"]:
    dict_temp["text"] = line[0]
    dict_temp["entities"]= line[1]["entities"]
    training_data.append(dict_temp)
    dict_temp={}


print(training_data)

[{'text': 'CANCELLATIONS\r\nANY TIME\r\nCHARGE USD 300.00 FOR CANCEL.\r\nNOTE  TEXT BELOW NOT VALIDATED FOR AUTOPRICING.\r\n\r\nFOR ALL JOURNEYS BEFORE AFTER FIRST FLIGHT\r\nDEPARTURE  USD300\r\n\r\nAPPLICABLE FEES ARE TO BE CONVERTED INTO\r\nLOCAL CURRENCY AT THE BANKERS SELLING RATE.\r\n\r\nFEES ARE NOT APPLICABLE FOR INFANT WITHOUT A SEAT\r\n\r\nIN CASE OF TICKET UPGRADE THE ORIGINAL\r\nNONREFUNDABLE AMOUNT REMAINS NONREFUNDABLE.\r\n\r\nYR IS NONREFUNDABLE FOR ALL FARES.\r\n\r\nIN CASES OF NOSHOW CHARGE NO SHOW FEE OF USD300\r\nAND CANCELLATION FEE OF USD300 FOR CANCEL.\r\n\r\nFOR ANY CANCELLATIONS THE STRICTER FARE\r\nCONDITIONS WILL APPLY TO THE ENTIRE JOURNEY.\r\n\r\n\r\n\r\n', 'entities': [[0, 23, 'CANCELLATION_TIME'], [25, 54, 'CHARGE_CANCEL'], [458, 486, 'CHARGE_NOSHOW']]}, {'text': '\r\nCANCELLATIONS\r\nANY TIME\r\nCHARGE USD 200.00 FOR CANCEL.\r\nNOTE  TEXT BELOW NOT VALIDATED FOR AUTOPRICING.\r\n\r\nFOR ALL JOURNEYS BEFORE or AFTER FIRST FLIGHT\r\nDEPARTURE  USD200\r\n\r\nA

Preparamos el entrenamiento

In [22]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

convertimos los datos a spacy.doc

In [23]:
from spacy.util import filter_spans

for training_example  in training_data: 
    text = training_example['text']
    labels = training_example['entities']
    print(labels)
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

[[0, 23, 'CANCELLATION_TIME'], [25, 54, 'CHARGE_CANCEL'], [458, 486, 'CHARGE_NOSHOW']]
[[2, 25, 'CANCELLATION_TIME'], [27, 56, 'CHARGE_CANCEL'], [463, 491, 'CHARGE_NOSHOW']]
[[2, 25, 'CANCELLATION_TIME'], [27, 56, 'CHARGE_CANCEL'], [463, 491, 'CHARGE_NOSHOW']]
[]
[[2, 25, 'CANCELLATION_TIME']]
[[2, 25, 'CANCELLATION_TIME']]
[[2, 25, 'CANCELLATION_TIME'], [560, 583, 'CANCELLATION_TIME']]
[[31, 53, 'CANCELLATION_TIME']]
[[17, 53, 'CANCELLATION_TIME']]
[[2, 25, 'CANCELLATION_TIME']]
[[17, 39, 'CANCELLATION_TIME'], [40, 69, 'CHARGE_CANCEL'], [460, 488, 'CHARGE_NOSHOW']]
[[17, 39, 'CANCELLATION_TIME'], [40, 69, 'CHARGE_CANCEL'], [460, 488, 'CHARGE_NOSHOW'], [513, 531, 'CHARGE_CANCEL']]
[[2, 33, 'CANCELLATION_TIME']]
[[2, 25, 'CANCELLATION_TIME']]
[[2, 25, 'CANCELLATION_TIME']]
[[2, 25, 'CANCELLATION_TIME']]
[[17, 40, 'CANCELLATION_TIME']]
[[17, 40, 'CANCELLATION_TIME']]
[[2, 25, 'CANCELLATION_TIME']]
[[2, 25, 'CANCELLATION_TIME']]
[[2, 25, 'CANCELLATION_TIME']]
[[17, 40, 'CANCELLATION_TIME'

aplicamos configuración para entrenar con GPU

In [24]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


lanzamos entrenamiento

In [25]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-09-20 17:31:45,900] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-09-20 17:31:45,910] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-09-20 17:31:45,914] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-09-20 17:31:45,915] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining m

guardamos los pesos

In [29]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [32]:
!cp -r /content/model-best /content/gdrive/MyDrive/Arquitecturas/

probamos el modelo resultante

In [5]:
from pydoc import doc
import spacy

nlp_ner = spacy.load("/content/model-best")

text = """
"CANCELLATIONS ANY TIME CHARGE USD 666.00 FOR CANCEL.
NOTE - TEXT BELOW NOT VALIDATED FOR AUTOPRICING.
FOR ALL JOURNEYS BEFORE/AFTER FIRST FLIGHT DEPARTURE - USD300
APPLICABLE FEES ARE TO BE CONVERTED INTO LOCAL CURRENCY AT THE BANKERS SELLING RATE.
FEES ARE NOT APPLICABLE FOR INFANT WITHOUT A SEAT
IN CASE OF TICKET UPGRADE THE ORIGINAL NON-REFUNDABLE AMOUNT REMAINS NON-REFUNDABLE.
YR IS NON-REFUNDABLE FOR ALL FARES.IN CASES OF NO-SHOW CHARGE NO SHOW FEE OF USD300 
AND CANCELLATION FEE OF USD300 FOR CANCEL.\
FOR ANY CANCELLATIONS THE STRICTER FARE\ CONDITIONS WILL APPLY TO THE ENTIRE JOURNEY.
"""

doc = nlp_ner(text)

colors = {
    "CANCELLATION_TIME": "#F67DE3",
    "CHARGE_CANCEL": "#7DF6D9",
    "CHARGE_NOSHOW": "#FFFFFF",
}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)
